# Merge NHANES 2007-08 data set
## Description
This code reads all the NHANES data files for a particular year (in SAS format) and store in a single csv file. Its uses the demographic file as the 'base' file to which all other data files are joine. It uses 'SEQN' as an index (formated as an integer). Multiple observations for the same SEQN are grouped and averaged. This occurs in a handful of Dietary file only.
## Inputs
NHANES SAS data files grouped by kind (Dietary, Examination, Lab, etc.)
## Output
One cvs file containing all the data for that year

In [1]:
import numpy as np
import pandas as pd
import glob 
import os
import sys

In [2]:
%cd /Users/elgayaro/Main/Research/Healthcare/CDC-NHANES/Data/07-08

/Users/elgayaro/Main/Research/Healthcare/CDC-NHANES/Data/07-08


In [3]:
data = pd.read_sas('demographic/DEMO_E.XPT', index='SEQN')
data.index = data.index.map(int)
print(data.shape)
data.head(10)

(10149, 42)


,SDDSRVYR,RIDSTATR,RIDEXMON,RIAGENDR,RIDAGEYR,RIDAGEMN,RIDAGEEX,RIDRETH1,DMQMILIT,DMDBORN2,...,FIAPROXY,FIAINTRP,MIALANG,MIAPROXY,MIAINTRP,AIALANG,WTINT2YR,WTMEC2YR,SDMVPSU,SDMVSTRA
SEQN,,,,,,,,,,,,,,,,,,,,,
41475,5.0,2.0,2.0,2.0,6.200000e+01,751.0,752.0,5.0,2.0,1.0,...,2.0,2.0,1.0,2.0,2.0,1.0,59356.356426,60045.772497,1.0,60.0
41476,5.0,2.0,1.0,2.0,6.000000e+00,81.0,82.0,5.0,NaN,1.0,...,2.0,2.0,NaN,NaN,NaN,NaN,35057.218405,35353.210440,1.0,70.0
41477,5.0,2.0,2.0,1.0,7.100000e+01,859.0,860.0,3.0,1.0,1.0,...,2.0,2.0,1.0,2.0,2.0,1.0,9935.266183,10074.150074,1.0,67.0
41478,5.0,2.0,2.0,2.0,1.000000e+00,17.0,17.0,3.0,NaN,1.0,...,2.0,2.0,NaN,NaN,NaN,NaN,12846.712058,14560.472652,2.0,59.0
41479,5.0,2.0,1.0,1.0,5.200000e+01,629.0,630.0,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,8727.797555,9234.055759,1.0,70.0
41480,5.0,2.0,1.0,1.0,6.000000e+00,81.0,81.0,1.0,NaN,1.0,...,2.0,2.0,NaN,NaN,NaN,NaN,7379.745086,7297.067503,2.0,69.0
41481,5.0,2.0,1.0,1.0,2.100000e+01,254.0,254.0,4.0,2.0,1.0,...,2.0,2.0,1.0,2.0,2.0,1.0,24342.505253,24655.376656,2.0,68.0
41482,5.0,2.0,2.0,1.0,6.400000e+01,778.0,779.0,1.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,9811.075078,11602.178638,2.0,65.0
41483,5.0,2.0,1.0,1.0,6.600000e+01,803.0,804.0,4.0,1.0,1.0,...,2.0,2.0,1.0,2.0,2.0,1.0,8058.685296,7920.812275,2.0,66.0


In [4]:
path = './Diatery'
allFiles = glob.glob(os.path.join(path,"*.XPT"))

print(allFiles)

print(data.shape)
for infile in allFiles:
    print(infile)
    df = pd.read_sas(infile, index='SEQN')
    df.index = df.index.map(int)
    print('file shape before grouping duplicates = ', df.shape)
    # df = df[~df.index.duplicated(keep='first')] # This will take the first obs. instead of the mean
    dfg = df.groupby('SEQN').mean()
    print('file shape after grouping duplicates = ', dfg.shape)
    fName = infile.replace(path + "/", "")
    fName = fName.replace("_E.XPT", "")
    data = data.join(dfg, how='left', rsuffix=fName)
    print(data.shape)
    print('====')

['./Diatery/DR1TOT_E.XPT', './Diatery/DSQTOT_E.XPT', './Diatery/DS2IDS_E.XPT', './Diatery/DR1IFF_E.XPT', './Diatery/DS1TOT_E.XPT', './Diatery/DR2TOT_E.XPT', './Diatery/DR2IFF_E.XPT', './Diatery/DSQIDS_E.XPT', './Diatery/DS2TOT_E.XPT', './Diatery/DS1IDS_E.XPT']
(10149, 42)
./Diatery/DR1TOT_E.XPT
file shape before grouping duplicates =  (9762, 163)
file shape after grouping duplicates =  (9762, 163)
(10149, 205)
====
./Diatery/DSQTOT_E.XPT
file shape before grouping duplicates =  (10149, 37)
file shape after grouping duplicates =  (10149, 37)
(10149, 242)
====
./Diatery/DS2IDS_E.XPT
file shape before grouping duplicates =  (5648, 45)
file shape after grouping duplicates =  (2662, 44)
(10149, 286)
====
./Diatery/DR1IFF_E.XPT
file shape before grouping duplicates =  (145703, 84)
file shape after grouping duplicates =  (9254, 84)
(10149, 370)
====
./Diatery/DS1TOT_E.XPT
file shape before grouping duplicates =  (9762, 46)
file shape after grouping duplicates =  (9762, 46)
(10149, 416)
====
.

In [5]:
path = './Lab'
allFiles = glob.glob(os.path.join(path,"*.XPT"))

print(allFiles)

print(data.shape)
for infile in allFiles:
    print(infile)
    df = pd.read_sas(infile, index='SEQN')
    df.index = df.index.map(int)
    print('file shape before grouping duplicates = ', df.shape)
    dfg = df.groupby('SEQN').mean()
    print('file shape after grouping duplicates = ', dfg.shape)
    fName = infile.replace(path + "/", "")
    fName = fName.replace("_E.XPT", "")
    data = data.join(dfg, how='left', rsuffix=fName)
    print(data.shape)
    print('====')

['./Lab/HEPB_S_E.XPT', './Lab/BIOPRO_E.XPT', './Lab/VOCWB_E.XPT', './Lab/TRIGLY_E.XPT', './Lab/APOB_E.XPT', './Lab/UHM_E.XPT', './Lab/HEPC_E.XPT', './Lab/TFR_E.XPT', './Lab/HEPBD_E.XPT', './Lab/DEET_E.XPT', './Lab/HEPA_E.XPT', './Lab/OPD_E.XPT', './Lab/SSUSG_E.XPT', './Lab/GHB_E.XPT', './Lab/PFC_E.XPT', './Lab/UPP_E.XPT', './Lab/PHTHTE_E.XPT', './Lab/VID_E.XPT', './Lab/FOLATE_E.XPT', './Lab/VOCMWB_E.XPT', './Lab/PBCD_E.XPT', './Lab/CRP_E.XPT', './Lab/VOC_E.XPT', './Lab/HDL_E.XPT', './Lab/TCHOL_E.XPT', './Lab/PAH_E.XPT', './Lab/FERTIN_E.XPT', './Lab/FASTQX_E.XPT', './Lab/VIT_B6_E.XPT', './Lab/EPH_E.XPT', './Lab/UAS_E.XPT', './Lab/UIO_E.XPT', './Lab/CARB_E.XPT', './Lab/HSV_E.XPT', './Lab/COTNAL_E.XPT', './Lab/CHLMDA_E.XPT', './Lab/GLU_E.XPT', './Lab/SSHCV_E.XPT', './Lab/HIV_E.XPT', './Lab/PSA_E.XPT', './Lab/HPVSER_E.XPT', './Lab/OGTT_E.XPT', './Lab/UCPREG_E.XPT', './Lab/PERNT_E.XPT', './Lab/UHG_E.XPT', './Lab/UPHOPM_E.XPT', './Lab/IHG_E.XPT', './Lab/HPVSWR_E.XPT', './Lab/UAM_E.XPT', './L

file shape before grouping duplicates =  (6917, 12)
file shape after grouping duplicates =  (6917, 12)
(10149, 1385)
====
./Lab/PP_E.XPT
file shape before grouping duplicates =  (2718, 12)
file shape after grouping duplicates =  (2718, 12)
(10149, 1397)
====
./Lab/FOLFMS_E.XPT
file shape before grouping duplicates =  (3122, 3)
file shape after grouping duplicates =  (3122, 3)
(10149, 1400)
====
./Lab/PHYTO_E.XPT
file shape before grouping duplicates =  (2718, 14)
file shape after grouping duplicates =  (2718, 14)
(10149, 1414)
====
./Lab/CBC_E.XPT
file shape before grouping duplicates =  (9307, 20)
file shape after grouping duplicates =  (9307, 20)
(10149, 1434)
====


In [6]:
path = './Quest'
allFiles = glob.glob(os.path.join(path,"*.XPT"))

print(allFiles)

print(data.shape)
for infile in allFiles:
    print(infile)
    df = pd.read_sas(infile, index='SEQN')
    df.index = df.index.map(int)
    print('file shape before grouping duplicates = ', df.shape)
    dfg = df.groupby('SEQN').mean()
    print('file shape after grouping duplicates = ', dfg.shape)
    fName = infile.replace(path + "/", "")
    fName = fName.replace("_E.XPT", "")
    data = data.join(dfg, how='left', rsuffix=fName)
    print(data.shape)
    print('====')

['./Quest/WHQ_E.XPT', './Quest/BHQ_E.XPT', './Quest/BPQ_E.XPT', './Quest/OCQ_E.XPT', './Quest/INQ_E.XPT', './Quest/CDQ_E.XPT', './Quest/DIQ_E.XPT', './Quest/ALQ_E.XPT', './Quest/SMQ_E.XPT', './Quest/CBQPFA_E.XPT', './Quest/SMQFAM_E.XPT', './Quest/PUQMEC_E.XPT', './Quest/DUQ_E.XPT', './Quest/CBQPFC_E.XPT', './Quest/ECQ_E.XPT', './Quest/SSQ_E.XPT', './Quest/CBQ_E.XPT', './Quest/FSQ_E.XPT', './Quest/PAQ_E.XPT', './Quest/OSQ_E.XPT', './Quest/MCQ_E.XPT', './Quest/IMQ_E.XPT', './Quest/HCQ_E.XPT', './Quest/HOQ_E.XPT', './Quest/KIQ_P_E.XPT', './Quest/ACQ_E.XPT', './Quest/SLQ_E.XPT', './Quest/DPQ_E.XPT', './Quest/VIQ_E.XPT', './Quest/AUQ_E.XPT', './Quest/SMQRTU_E.XPT', './Quest/SXQ_E.XPT', './Quest/PFQ_E.XPT', './Quest/HUQ_E.XPT', './Quest/AQQ_E.XPT', './Quest/WHQMEC_E.XPT', './Quest/RXQ_RX_E.XPT', './Quest/OHQ_E.XPT', './Quest/HIQ_E.XPT', './Quest/HSQ_E.XPT', './Quest/KIQ_U_E.XPT', './Quest/DBQ_E.XPT', './Quest/RHQ_E.XPT', './Quest/RDQ_E.XPT']
(10149, 1434)
./Quest/WHQ_E.XPT
file shape before 

In [7]:
path = './Examination'
allFiles = glob.glob(os.path.join(path,"*.XPT"))

print(allFiles)

print(data.shape)
for infile in allFiles:
    print(infile)
    df = pd.read_sas(infile, index='SEQN')
    df.index = df.index.map(int)
    print('file shape before grouping duplicates = ', df.shape)
    dfg = df.groupby('SEQN').mean()
    print('file shape after grouping duplicates = ', dfg.shape)
    fName = infile.replace(path + "/", "")
    fName = fName.replace("_E.XPT", "")
    data = data.join(dfg, how='left', rsuffix=fName)
    print(data.shape)
    print('====')

['./Examination/DXXFEM_E.XPT', './Examination/OPXRET_E.XPT', './Examination/AUX_E.XPT', './Examination/VIX_E.XPT', './Examination/OHX_E.XPT', './Examination/ENX_E.XPT', './Examination/SPX_E.XPT', './Examination/AUXAR_E.XPT', './Examination/BMX_E.XPT', './Examination/BPX_E.XPT', './Examination/OPXFDT_E.XPT', './Examination/DXXSPN_E.XPT', './Examination/AUXTYM_E.XPT']
(10149, 2568)
./Examination/DXXFEM_E.XPT
file shape before grouping duplicates =  (7734, 19)
file shape after grouping duplicates =  (7734, 19)
(10149, 2587)
====
./Examination/OPXRET_E.XPT
file shape before grouping duplicates =  (3863, 221)
file shape after grouping duplicates =  (3863, 221)
(10149, 2808)
====
./Examination/AUX_E.XPT
file shape before grouping duplicates =  (1210, 70)
file shape after grouping duplicates =  (1210, 70)
(10149, 2878)
====
./Examination/VIX_E.XPT
file shape before grouping duplicates =  (6917, 56)
file shape after grouping duplicates =  (6917, 56)
(10149, 2934)
====
./Examination/OHX_E.XPT
f

In [8]:
data.to_csv('07_08_ofg_grpDup.csv')